In [ ]:
# -*- coding: utf-8 -*-
"""
==============================================================================
@author: Nikolaos Giakoumoglou
@date: Thu May  6 19:10:11 2021
@reference: Weszka, A Comparative Study of Texture Measures for Terrain Classification
==============================================================================
"""

import numpy as np


def glds(f, mask, dx, dy, Ng):
    '''
    Parameters
    ----------
    f : numpy ndarray
        Image of dimensions N1 x N2.
    mask : numpy ndarray
        Mask image N1 x N2 with 1 if pixels belongs to ROI, 0 else
    dx : int
        Orientation in X-coordinate
    dy : int
        Orientation in Y-coordinate
    Ng : int
        Image number of gray values

    Returns
    -------
    f_d : numpy ndarray
    p_d : numpy ndarray
    '''

    N1, N2 = f.shape

    # Calculate f_d(x,y). If calculation includes pixel outside mask, ignore it.
    f_d = np.zeros((N1, N2), np.double)
    for x in range(N1):
        for y in range(N2):
            if (x+dx < N1) & (y+dy < N2) & (x+dx >= 0) & (y+dy >= 0) & (mask[x, y] == 1):
                f_d[x, y] = abs(f[x, y] - f[x+dx, y+dy])

    # Calculate pd_(i)
    f_d_ravel = f_d.ravel()
    mask_ravel = mask.ravel()
    roi = f_d_ravel[mask_ravel.astype(bool)]
    p_d = np.histogram(roi, bins=Ng, range=(0, Ng-1)
                       )[0]  # histogram of f_d in ROI

    return f_d, p_d


def glds_features(f, mask, Dx=[0, 1, 1, 1], Dy=[1, 1, 0, -1]):
    '''
    Parameters
    ----------
    f : numpy ndarray
        Image of dimensions N1 x N2.
    mask : numpy ndarray
        Mask image N1 x N2 with 1 if pixels belongs to ROI, 0 else.
    Dx : int, optional
        Array with X-coordinates of vectors denoting orientation. The default
        is [0,1,1,1].
    Dy : int, optional
        Array with Y-coordinates of vectors denoting orientation. The default
        is [1,1,0,-1].

    Returns
    -------
    features : numpy ndarray
        1) Contrast, 2)Angular Second Moment, 3)Entropy, 4)Mean
    labels : list
        Labels of features.
    '''

    if mask is None:
        mask = np.ones(f.shape)

    # 1) Labels
    labels = ["GLDS_Homogeneity", "GLDS_Contrast",
              "GLDS_ASM", "GLDS_Entopy", "GLDS_Mean"]

    # 2) Parameters
    f = f.astype(np.double)
    mask = mask.astype(np.double)
    Dx = np.array(Dx)
    Dy = np.array(Dy)
    Ng = 256

    # 3) Loop over Dx, Dy values to calculate feats
    features = []
    for ii in range(Dx.shape[0]):

        dx = Dx[ii]
        dy = Dy[ii]

        f_d, p_d = glds(f, mask, dx, dy, Ng)

        feats = np.zeros(5, np.double)
        i = np.arange(Ng)
        i2 = i ** 2
        feats[0] = sum(np.divide(p_d, i2+1))
        feats[1] = sum(np.multiply(p_d, i2))
        feats[2] = sum(np.multiply(p_d, p_d))
        feats[3] = -sum(np.multiply(p_d, np.log(p_d+1e-16)))
        feats[4] = sum(np.multiply(p_d, i))
        features.append(feats)

    # 4) Calculate Features: mean of feats
    features = np.array(features)
    features = features.mean(axis=0)

    return features, labels


In [6]:
import numpy as np

features = []

f =np.array([[0,0,1,1],
             [0,0,1,1],
             [0,2,2,2],
             [2,2,3,3]])

mask = None
if mask is None:
    mask = np.ones(f.shape)
m = f.astype(np.double)
Dx=[0]
Dy=[1]
Dx = np.array(Dx)
Dy = np.array(Dy)
Ng = 4

print(f)

for ii in range(Dx.shape[0]):

    dx = Dx[ii]
    dy = Dy[ii]
    print (dx, dy)
    print("=" * 100)

    N1, N2 = f.shape

    # Calculate f_d(x,y). If calculation includes pixel outside mask, ignore it.
    f_d = np.zeros((N1, N2), np.double)
    for x in range(N1):
        for y in range(N2):
            if (x+dx < N1).any() & (y+dy < N2).any() & (x+dx >= 0).any() & (y+dy >= 0).any() & (mask[x, y] == 1):
                a = f[x, y]
                da = f[x+dx, y+dy] 
                f_d[x, y] = abs(a - da)
                print ("a  : "+ str(a))
                print ("da : " + str(da))
                print ("f_d[x, y] = " + str(abs(a - da)))
                print("=" * 25)

    # Calculate pd_(i)
    f_d_ravel = f_d.ravel()
    mask_ravel = mask.ravel()
    roi = f_d_ravel[mask_ravel.astype(bool)]
    p_d = np.histogram(roi, bins=Ng, range=(0, Ng-1)
                       )[0]  # histogram of f_d in ROI
    print(f_d)
    print(f_d_ravel)
    # print("=======================================")
    print(mask_ravel)
    print(f'roi {roi}')
    print(f'pd = {p_d}')

    feats = np.zeros(5, np.double)
    i = np.arange(Ng)
    i2 = i ** 2
    feats[0] = sum(np.divide(p_d, i2+1)) # Homogeneity
    feats[1] = sum(np.multiply(p_d, i2)) # Contrast
    feats[2] = sum(np.multiply(p_d, p_d)) # ASM
    feats[3] = -sum(np.multiply(p_d, np.log(p_d+1e-16))) # entropy
    feats[4] = sum(np.multiply(p_d, i)) # Mean
    features.append(feats)

    # 4) Calculate Features: mean of feats
features = np.array(features)
    # features = features.mean(axis=0)
# # Calculate pd_(i)
# f_d_ravel = f_d.ravel()
# mask_ravel = mask.ravel()
# roi = f_d_ravel[mask_ravel.astype(bool)]
# p_d = np.histogram(roi, bins=Ng, range=(0, Ng-1)
#                     )[0]  # histogram of f_d in ROI

[[0 0 1 1]
 [0 0 1 1]
 [0 2 2 2]
 [2 2 3 3]]
0 1
a  : 0
da : 0
f_d[x, y] = 0
a  : 0
da : 1
f_d[x, y] = 1
a  : 1
da : 1
f_d[x, y] = 0
a  : 0
da : 0
f_d[x, y] = 0
a  : 0
da : 1
f_d[x, y] = 1
a  : 1
da : 1
f_d[x, y] = 0
a  : 0
da : 2
f_d[x, y] = 2
a  : 2
da : 2
f_d[x, y] = 0
a  : 2
da : 2
f_d[x, y] = 0
a  : 2
da : 2
f_d[x, y] = 0
a  : 2
da : 3
f_d[x, y] = 1
a  : 3
da : 3
f_d[x, y] = 0
[[0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [2. 0. 0. 0.]
 [0. 1. 0. 0.]]
[0. 1. 0. 0. 0. 1. 0. 0. 2. 0. 0. 0. 0. 1. 0. 0.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
roi [0. 1. 0. 0. 0. 1. 0. 0. 2. 0. 0. 0. 0. 1. 0. 0.]
pd = [12  3  1  0]


In [3]:
features

array([[ 13.7       ,   7.        , 154.        , -33.11471666,
          5.        ]])

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
fig.add_subplot(1,2, 1)
plt.imshow(f, cmap='gray')
fig.add_subplot(1,2, 2)
plt.imshow(f_d, cmap='gray')
plt.show(block=True)

In [12]:
a  = [1, 2, 3, 4]
b =  len(a)

penampung = [0]

def coba():
  for i in range(b):
    penampung.clear()
    penampung.append(i)

  for dir in range(b):
    # penampung.clear()
    # penampung.append(dir)
    # print("="*10)
    # print(penampung[0])
    yield dir
    yield 2

c = coba()
print(next(c))
penampung[0]


0


3

In [ ]:
from _mahotas.features.texture import haralick

glcm2 = haralick(f, compute_14th_feature=False, ignore_zeros=True, return_mean=True)
glcm2

In [1]:
from _features._selected import features_selected
import cv2
list = []
image = cv2.imread('Images/Groundtruth/Negatif/1/Left Crowsfeet1.jpg', cv2.IMREAD_GRAYSCALE)
list.append(image)

features = features_selected.selected_extract_texture(list)

# features_fos, labels = fos.FirstOrderStatistic(image, mask = None)
# print(features_fos)
# print(labels)

In [9]:
from _mahotas.features.texture_four_dir import *
import numpy as np

glcm = haralick(image, compute_14th_feature=True, ignore_zeros=True)
# glcm = np.hstack(glcm)
# flcm = [i for i in glcm if i != 0]

print(glcm)


[[ 2.65654044e-03  4.20737849e+00  0.00000000e+00  0.00000000e+00
   5.03867785e-01  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  1.98634345e-03  2.24765323e+00  0.00000000e+00
   9.99509119e-01  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  5.81324113e+02
   0.00000000e+00  1.61715808e+02  0.00000000e+00  0.00000000e+00
   0.00000000e+00  1.65513954e-03  0.00000000e+00  0.00000000e+00
   0.00000000e+00  2.99673567e+00]
 [ 0.00000000e+00  0.00000000e+00  9.95861373e-01  0.00000000e+00
   0.00000000e+00  1.61770693e+02  2.32911199e+03  0.00000000e+00
   0.00000000e+00  1.88867445e-03  0.00000000e+00 -5.29435061e-01
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  2.84680715e+04  7.38723201e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00 -4.65025831e-01
   0.00000000e+00  3.47828645e+00]]


In [3]:
# cmats = []
# for dir in range(4):
#   u = cooccurence(image, dir)
#   cmats.append(u)
# # u
# for (i, item) in enumerate(cmats, start=1):
#     print(i, item)

In [4]:
import numpy as np

a = features.GLCM_four_dir
a
# np.ravel(a)


[[0.0026565404358660973,
  4.207378486055778,
  0.5038677849313471,
  0.0019863434528196986,
  2.247653226736494,
  0.9995091187627375,
  581.3241127267838,
  161.71580799999998,
  0.0016551395397500013,
  2.996735670975729,
  0.9958613726512013,
  161.7706932270916,
  2329.111992109083,
  0.0018886744515340045,
  -0.5294350612519374,
  28468.071519999998,
  7.387232013495241,
  -0.46502583069568987,
  3.4782864500215642]]

In [4]:
features.Labels_GLCM_four_dir

['GLCM_ASM_0',
 'GLCM_Contrast_0',
 'GLCM_InverseDifferenceMoment_0',
 'GLCM_DifferenceVariance_0',
 'GLCM_DifferenceEntropy_0',
 'GLCM_Information2',
 'GLCM_SumOfSquaresVariance_45',
 'GLCM_SumAverage_45',
 'GLCM_DifferenceVariance_45',
 'GLCM_MaximalCorrelationCoefficient_45',
 'GLCM_Correlation_90',
 'GLCM_SumAverage_90',
 'GLCM_SumVariance_90',
 'GLCM_DifferenceVariance_90',
 'GLCM_Information1_90',
 'GLCM_SumVariance_135',
 'GLCM_SumEntropy_135',
 'GLCM_Information1_135',
 'GLCM_MaximalCorrelationCoefficient_135']

In [2]:
from __utils__.convert2dataframe import *

df_train = convert2dataframe(features)

df_result_training = pd.concat([df_train.fos, 
                                df_train.GLCM_mean, 
                                df_train.GLCM_four_dir,
                                df_train.GLDS, 
                                df_train.GLRLM
                                ], axis=1)
df_result_training


,FOS_StandardDeviation,FOS_Skewness,FOS_MinimalGrayLevel,FOS_10Percentile,FOS_Smoothness,Contrast_Mean,Inverse Difference Moment_Mean,Information Measure of Correlation 1_Mean,Maximal Correlation Coefficient_Mean,GLCM_ASM_0,...,GLCM_Information1_90,GLCM_SumVariance_135,GLCM_SumEntropy_135,GLCM_Information1_135,GLCM_MaximalCorrelationCoefficient_135,GLDS_Contrast,GLRLM_LongRunEmphasis,GLRLM_RunPercentage,GLRLM_HighGrayLevelRunEmphasis,GLRLM_ShortRunHighGrayLevelEmphasis
0,24.177507,-0.000078,5.0,45.0,0.998292,6.082107,0.45571,-0.507401,3.083024,0.002657,...,-0.529435,28468.07152,7.387232,-0.465026,3.478286,380696.5,2.127776,0.764917,6935.484413,5772.027272


In [ ]:
glcm3 = haralick(image, compute_14th_feature=True, ignore_zeros=True, return_mean=True)
glcm3


In [ ]:
from _mahotas.features.texture import haralick_labels

print(haralick_labels)

In [ ]:
labels_0 = ["GLCM_ASM_0",
            "GLCM_Contrast_0",
            "GLCM_InverseDifferenceMoment_0",
            "GLCM_DifferenceVariance_0",
            "GLCM_DifferenceEntropy_0",
            "GLCM_Information2"]
labels_45 = ["GLCM_SumOfSquaresVariance_45",
              "GLCM_SumAverage_45",
              "GLCM_DifferenceVariance_45",
              "GLCM_MaximalCorrelationCoefficient_45"]
labels_90 = ["GLCM_Correlation_90",
              "GLCM_SumAverage_90",
              "GLCM_SumVariance_90",
              "GLCM_DifferenceVariance_90",
              "GLCM_Information1_90"]
labels_135 = ["GLCM_SumVariance_135",
              "GLCM_SumEntropy_135", 
              "GLCM_Information1_135",
              "GLCM_MaximalCorrelationCoefficient_135"]
labelss = []
labelss.extend(labels_0)
labelss.extend(labels_45)
labelss.extend(labels_90)
labelss.extend(labels_135)
len(labelss)

In [ ]:
_2d_deltas = [
    (0, 1),
    (1, 1),
    (1, 0),
    (1, -1)]

a = [0]
def num(dir):
  if a[0]== None: 
    a.append(dir)

def cek():
  
  for dir in range(len(_2d_deltas)):
    a.clear()
    a.append(dir)
  # # return a
    yield _2d_deltas

dir = cek()
# print(a)


for i in dir:
  print(i)
  print(a)
  # print(a)

# next(dir)


In [3]:
import pandas as pd

tr = pd.read_csv('Datasets Testing.csv')
tr.to_excel('Datasets Testing.xlsx', index = False)